# P3 Avistamiento de aves

## Obtener datos (scraping)

Primero vamos a obtener los datos. Para ello nos visitaremos las web de avistamientos de aves [shorebirder](https://www.shorebirder.com/), [trevorsbirding](https://www.trevorsbirding.com/) y [dantallmansbirdblog](https://dantallmansbirdblog.blogspot.com/).

Durante la visita a la web y haciendo uso del inspector (F12) podemos ver que las descripciones que necesitamos se encuentran en los tag de párrafo (entre *\<p\> TEXTO \</p\>*). Sabiendo eso vamos a crear funciones de utilidad que se encargarán de descargar el contenido de la web y extraer el texto.

Las descargas las realizaremos en `data/raw` mientras que en `data/posts` guardaremos los textos encontrados.

** `dantallmansbirdblog` tiene una estructura ligeramente diferente (entre *\<p\>\</p\> TEXTO \<p\>\</p\>*), a lo que tendremos que modificar la función `get_texts` (a continuación) para obtener sus textos.

In [1]:
import os
import re
import wget

data_posts_cache = "../data/cache" # guardar resultados de queries a sparql
data_raw_path = "../data/raw" # descargas
data_posts_path = "../data/posts" # guardar los textos de los post scrapeados

def maybe_mkdir(path):
  try:
    os.mkdir(path)
  except OSError as error:
    print(error)

maybe_mkdir("../data")
maybe_mkdir(data_raw_path)
maybe_mkdir(data_posts_path)

def download(url, out_label):
  return wget.download(url, out=f"{data_raw_path}/{out_label}")

def get_texts(filename):
  file = open(filename, 'r')
  text = file.read()
  file.close()

  # get texts
  get_p = re.compile(r'<p>((.|\n)*?)</p>')
  texts = get_p.findall(text)

  # remove styling and inner tags
  remove_tags = re.compile(r'(<.*?>)|\\n| +(?= )|\\|\&.+?\;')
  return map(lambda text: re.sub(remove_tags, "", str(text[0]).lower()), texts)

def write(path, filename, data):
  filepath = f"{path}/{filename}.txt"
  file = open(filepath, "a", encoding="utf-8")
  for item in data:
    file.write(str(item)+"\n")
  file.close()
  return filepath


A continuación haciendo uso de las funciones anteriores scrapeamos la home de `shorebirder`.

In [2]:
shorebirder_filename = "shorebirder_home.html"
shorebirder_home = download("https://www.shorebirder.com/", shorebirder_filename)
posts = get_texts(shorebirder_home)
shorebirder_posts_file = write(data_posts_path, shorebirder_filename, posts)

open(shorebirder_posts_file, "r").readlines()[0]

"my late march solo visit to norway is in the books and was about as much fun as i've had in a while. the middle few days of the trip were spent birding around varanger, bookended by more touristy time intromsÃ£Â¸and oslo. at some point in the coming months there will be a full trip report here plus a very detailed cloudbirders submission. in the meantime, here is some proof that i actually went.\n"

Lo mismo para `trevorsbirding`.

In [3]:
trevorsbirding_filename = "trevorsbirding_home.html"
trevorsbirding_home = download("https://www.trevorsbirding.com/", trevorsbirding_filename)
posts = get_texts(trevorsbirding_home)
trevorsbirding_posts_file = write(data_posts_path, trevorsbirding_filename, posts)

open(trevorsbirding_posts_file, "r").readlines()[2]

'earlier this week i glanced out of my sunroom window to check whether there were any birds at my birdbaths. i currently have three birdbaths just outside the room, one on the ground, one on a pedestal at about 60cm and one hanging from a tree branch at a height of about 1.5 metres. i was delighted to see a small flock of purple-crowned lorikeets having a drink and dipping into the water for a bath. i have just checked my list of species to have visited the birdbaths. this was bird species number 36, in addition to the three reptiles and two mammal species.\n'

## Analizar articulos

In [4]:
# Cargar dependencias para el nlp
import spacy
from collections import Counter
from spacy import displacy
spacy.prefer_gpu()
nlp = spacy.load("en_core_web_lg")

In [6]:
for text in open(shorebirder_posts_file, "r").readlines()[3:5]:
  train = nlp(text)
  displacy.render(train, jupyter=True, style="ent")

c:\tools\Anaconda3\lib\site-packages\spacy\displacy\__init__.py:205: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)


Este nlp no nos sirve o bien nos falta entrenarlo. No es capaz de encontrar los nombres de pájaros.

vamos a probar de entrenarlo usando nombres de pájaros de la dbpedia.

In [7]:
# Prefixes and Class based from https://github.com/ejrav/pydbpedia
from SPARQLWrapper import SPARQLWrapper, JSON

class SparqlEndpoint(object):

    def __init__(self, endpoint, prefixes={}):
        self.sparql = SPARQLWrapper(endpoint)
        self.prefixes = {
            "dbo": "http://dbpedia.org/ontology/",
            "owl": "http://www.w3.org/2002/07/owl#",
            "xsd": "http://www.w3.org/2001/XMLSchema#",
            "rdfs": "http://www.w3.org/2000/01/rdf-schema#",
            "rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
            "foaf": "http://xmlns.com/foaf/0.1/",
            "dc": "http://purl.org/dc/elements/1.1/",
            "dbpedia2": "http://dbpedia.org/property/",
            "dbpedia": "http://dbpedia.org/",
            "skos": "http://www.w3.org/2004/02/skos/core#",
            "foaf": "http://xmlns.com/foaf/0.1/",
            "yago": "http://dbpedia.org/class/yago/",
            }
        self.prefixes.update(prefixes)
        self.sparql.setReturnFormat(JSON)

    def query(self, q):
        lines = ["PREFIX %s: <%s>" % (k, r) for k, r in self.prefixes.items()]
        lines.extend(q.split("\n"))
        query = "\n".join(lines)
        self.sparql.setQuery(query)
        results = self.sparql.query().convert()
        return results["results"]["bindings"]


class DBpediaEndpoint(SparqlEndpoint):
    def __init__(self, endpoint, prefixes = {}):
        super(DBpediaEndpoint, self).__init__(endpoint, prefixes)

In [8]:
s = DBpediaEndpoint(endpoint = "http://dbpedia.org/sparql")

birds = s.query("""
  SELECT DISTINCT *
  WHERE {
    ?bird a dbo:Bird ;
          a yago:Bird101503061 ;
          rdfs:label ?name ;
          dbo:abstract ?comment .

    filter (!isLiteral(?name) ||
            langmatches(lang(?name), "en")) .

    filter (!isLiteral(?comment) ||
            langmatches(lang(?comment), "en")) .
    
  }
  limit 10000
""")

write("../data", "birds", birds)

print(f"Hemos obtenido los nombres de {len(birds)} pájaros")
for d in birds[0:5]:
  print(d['name']['value'])

Hemos obtenido los nombres de 7330 pájaros
African pitta
African red-eyed bulbul
Andean coot
Angolan lark
Antillean crested hummingbird


In [28]:
import unicodedata
def strip_accents(s):
   return ''.join(c for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn')

# build training sentences
training_data = [
  # ("Tokyo Tower is 333m tall.", [(0, 11, "BUILDING")]), # example
]

tag = "BIRD"

remove_parentesis_text = re.compile(r'\(.*\)')

# mejorable pero hace su trabajo
for bird in birds:
  bird_name = bird['name']['value']
  if len(bird_name) == 0:
    continue
  
  bird_name = re.sub(remove_parentesis_text, "", str(bird_name).lower())
  train_sentence = bird['comment']['value']
  try:
    start_at = strip_accents(train_sentence).lower().index(strip_accents(bird_name))
    ends_at = start_at + len(bird_name)
    training_data.append(
      (bird_name, [(0, len(bird_name), tag)])
    )
    training_data.append(
      (train_sentence, [(start_at, ends_at, tag)])
    )
  except:
    # print("Error: substing not found")
    # print(bird_name)
    # print(train_sentence)
    # print("-"*10)
    training_data.append(
      (bird_name, [(0, len(bird_name), tag)])
    )

for text, annotations in training_data[0:10]:
  print(text)
  print(annotations)

african pitta
[(0, 13, 'BIRD')]
african red-eyed bulbul
[(0, 23, 'BIRD')]
andean coot
[(0, 11, 'BIRD')]
angolan lark
[(0, 12, 'BIRD')]
antillean crested hummingbird
[(0, 29, 'BIRD')]
antillean mango
[(0, 15, 'BIRD')]
arabian bustard
[(0, 15, 'BIRD')]
asian barred owlet
[(0, 18, 'BIRD')]
balicassiao
[(0, 11, 'BIRD')]
band-tailed oropendola
[(0, 22, 'BIRD')]


In [29]:
import spacy
from spacy.tokens import DocBin
from spacy.util import filter_spans

nlp = spacy.blank("en")
skips = 0
# the DocBin will store the example documents
db = DocBin()
for text, annotations in training_data:
  doc = nlp(text)
  ents = []
  for start, end, label in annotations:
    span = doc.char_span(start, end, label=label, alignment_mode="contract")
    if span is None:
      skips += 1
    else:
      ents.append(span)
  filtered_ents = filter_spans(ents)
  doc.ents = filtered_ents
  db.add(doc)

print(f"Skipped {skips} entries")
db.to_disk("./birds_train.spacy")

Skipped 0 entries


In [32]:
# train the model
!python -m spacy init fill-config birds_config.cfg config.cfg
!python -m spacy train config.cfg --output ./output --paths.train ./birds_train.spacy --paths.dev ./dev.spacy # dev.spacy it's the validation set ... TODO

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy
ℹ Saving to output directory: output
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
⚠ Aborting and saving the final best model. Encountered exception:
FileNotFoundError(2, 'No such file or directory')


[2022-05-10 01:07:40,029] [INFO] Set up nlp object from config
[2022-05-10 01:07:40,041] [INFO] Pipeline: ['tok2vec', 'ner']
[2022-05-10 01:07:40,047] [INFO] Created vocabulary
[2022-05-10 01:07:40,049] [INFO] Finished initializing nlp object
[2022-05-10 01:07:42,060] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
Traceback (most recent call last):
  File "c:\tools\Anaconda3\lib\runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\tools\Anaconda3\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "c:\tools\Anaconda3\lib\site-packages\spacy\__main__.py", line 4, in <module>
    setup_cli()
  File "c:\tools\Anaconda3\lib\site-packages\spacy\cli\_util.py", line 71, in setup_cli
    command(prog_name=COMMAND)
  File "c:\tools\Anaconda3\lib\site-packages\click\core.py", line 1128, in __call__
    return self.main(*args, **kwargs)
  File "c:\tools\Anaconda3\lib\site-packages\click\core.py", line 1053, in main
 

In [37]:
my_nlp = spacy.load("./output/model-best")
for text in open(shorebirder_posts_file, "r").readlines()[4:5]:
  doc = my_nlp(text)
  if len(doc.ents) > 0:
    displacy.render(doc, jupyter=True, style="ent")

c:\tools\Anaconda3\lib\site-packages\spacy\displacy\__init__.py:133: UserWarning: [W005] Doc object not parsed. This means displaCy won't be able to generate a dependency visualization for it. Make sure the Doc was processed with a model that supports dependency parsing, and not just a language class like `English()`. For more info, see the docs:
https://spacy.io/usage/models
  warnings.warn(Warnings.W005)
